In [1]:
import pandas as pd
import numpy as np

In [12]:
raw_data = pd.read_csv('empresasEafit.csv')
raw_data.rename(columns=lambda x: x.strip(), inplace=True)
raw_data.head()

,Razón social,Macrosector,"Tipo de propiedad (Privada, Pública, Mixta)",¿Es empresa familiar? Si/No,¿Cotiza en bolsa? Si/No,¿Multinacional? Si/No,Año de fundación,País de base principal (nacionalidad),Ingresos operacionales,Variable,Valor,Bloque,Nombre Pilar,Ponderación Materialidad,Valoración,valoracionPonderada
0,Aliar,Agropecuario,Privada,No,No,No,2007,Colombia,0.97802211500000003,¿Cuál es el porcentaje de proveeduría local?,No informa,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",14%,0.000000,0.000000
1,Alimentos Carnicos (Grupo Nutresa),Manufactura,Privada,No,Si,Si,1920,Colombia,2.844632077,¿Cuál es el porcentaje de proveeduría local?,0.80000000000000004,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",11%,84.210526,8.864266
2,Alpina,Manufactura,Privada,No,No,Si,1945,Colombia,2.1764613960000001,¿Cuál es el porcentaje de proveeduría local?,0.91000000000000003,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",11%,95.789474,10.083102
3,Amarilo,Construcción,Privada,Si,No,No,1993,Colombia,2.1863011939999999,¿Cuál es el porcentaje de proveeduría local?,0.53000000000000003,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",14%,53.000000,7.571429
4,ARA,Comercio,Privada,No,No,Si,2012,Colombia,13.83118268,¿Cuál es el porcentaje de proveeduría local?,0,Impacto económico y gobernanza,"Impacto Económico, Social y Comunitario",12%,0.000000,0.000000


# **Parte 1: Panorama General del Desempeño Corporativo**

## 1. Radar de Competitividad por Pilar y Macrosector

### Manejo de datos

In [13]:
# Encadenar las operaciones de pandas para simplificar el proceso:
# 1. Agrupar por Macrosector y Nombre Pilar, calculando la media.
# 2. Pivotar la tabla para obtener la estructura deseada (Pilares como filas, Sectores como columnas).
# 3. Rellenar los valores nulos con 0.
# 4. Resetear el índice para que 'Nombre Pilar' vuelva a ser una columna.
# 5. Renombrar 'Nombre Pilar' a 'category' para compatibilidad con amCharts.
radar_final_df = (
    raw_data.groupby(['Macrosector', 'Nombre Pilar'])['valoracionPonderada']
    .mean()
    .reset_index()
    .pivot_table(
        index='Nombre Pilar',
        columns='Macrosector',
        values='valoracionPonderada'
    )
    .fillna(0)
    .reset_index()
    .rename(columns={'Nombre Pilar': 'category'})
)

# --- PASO 3: Guardar los datos en formato JSON ---
radar_final_df.to_json('Dashboard/radar_data.json', orient='records', indent=4, force_ascii=False)


## 2. Matriz Térmica de Desempeño Empresaria

### Manejo de datos

In [18]:
# 1. Pivotar la tabla para agregar los valores.
#    - Se agrupa por 'Razón social' y 'Nombre Pilar'.
#    - Se calcula la SUMA de 'valoracionPonderada' como indica la guía.
#    - Se rellenan los valores nulos con 0.
heatmap_pivot = raw_data.pivot_table(
    index='Razón social',
    columns='Nombre Pilar',
    values='valoracionPonderada',
    aggfunc='sum',
    fill_value=0
)

# 2. "Derretir" (unpivot) la tabla para convertirla a un formato largo.
#    Este es el formato que amCharts consume más fácilmente para un heatmap.
#    Cada fila representará una celda única en el mapa de calor.
heatmap_long = heatmap_pivot.reset_index().melt(
    id_vars='Razón social',
    var_name='pilar',
    value_name='value'
)

# Renombrar 'Razón social' para mayor claridad en el JSON
heatmap_long.rename(columns={'Razón social': 'empresa'}, inplace=True)
# Ordenar empresa de manera ascendente
heatmap_long.sort_values(by='empresa', inplace=True)


# --- PASO 3: Guardar los datos en formato JSON ---

# Guardar el DataFrame largo en un archivo JSON.
heatmap_long.to_json('Dashboard/heatmap_data.json', orient='records', indent=4, force_ascii=False)


## 3. Diagrama de Violín de la Dispersión del Desempeño Sectorial

### Manejo de datos